## Ollama inference to structured json

Later
* Develop script for pre-training on json instructions
* Develop script for LORA fine-tuning on json instructions
* Test fine-tuning Llama on json instruction

In [18]:
import os
from ollama import chat
from pydantic import BaseModel

# os.sys.path.append(r"C:\\Code\\SDev.Python") # Path to root above sdevpy

# Global setup
root_folder = r"C:\\temp\\llms"
data_folder = os.path.join(root_folder, "datasets")

### Extract json information from email
Read an email in a text file. Define a json schema for the output. Give instruction to the model to extract information from the input email in the json format.

In [ ]:
file = os.path.join(data_folder, "customer_support.txt")
with open(file, "r", encoding='utf-8') as f:
    email = f.read()

print(email)

Subject: Issue with Recent Order #48291

From: emma.johnson@example.com

To: support@shopfast.com

Date: October 26, 2025

Hi ShopFast team,

I placed an order (Order #48291) on October 20, but the package hasn’t arrived yet at 456 Kennedy Ave, 121489 Atlanta, even though the tracking page says “Delivered” since October 23. Could you please check what happened?

Also, I was charged twice for this order on my credit card. Please confirm if I’ll get a refund for the duplicate charge.

Thanks,
Emma Johnson
+44 7911 123456


In [ ]:
# Define json schema
class OutputSchemaModel(BaseModel):
    customer_name: str
    phone_number: str
    order_number: str
    delivery_address: str

output_schema = OutputSchemaModel.model_json_schema()
print(output_schema)

{'properties': {'customer_name': {'title': 'Customer Name', 'type': 'string'}, 'phone_number': {'title': 'Phone Number', 'type': 'string'}, 'order_number': {'title': 'Order Number', 'type': 'string'}, 'delivery_address': {'title': 'Delivery Address', 'type': 'string'}}, 'required': ['customer_name', 'phone_number', 'order_number', 'delivery_address'], 'title': 'OutputSchemaModel', 'type': 'object'}


In [39]:
model_name = 'llama3.2:1b'

response = chat(model=model_name, messages=[
  {
    'role': 'user',
    'content': f'Please extract json information from the following: {email}',
  },]
  , format=output_schema)

print(response['message']['content'])

{ "customer_name": "Emma Johnson", "phone_number": "+44 7911 123456", "order_number": "48291", "delivery_address": "456 Kennedy Ave, 121489 Atlanta, USA" }
